In [4]:
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [5]:
os.chdir("Q:\\F_Train")
columns = np.arange(1,785)
data = pd.DataFrame()
for dirc in tqdm(os.listdir(),desc="Loading..."):
    path = "Q:\\F_Train\\{}".format(dirc)
    list = []
    for f in os.listdir(path):
        path = "Q:\\F_Train\\{}\\{}".format(dirc,f)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(28,28)).ravel()
        list.append(img)
    df = pd.DataFrame(columns=columns)
    for arr in list:
        l = len(df)
        df.loc[l]=arr
    df.insert(0,'Type',int(dirc))
    data = pd.concat([data,df],axis=0)

Loading...: 100%|███████████████████████████████████████████████████████████████████████████████████| 32/32 [06:42<00:00, 12.58s/it]


In [6]:
data.describe()

,Type,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
count,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,...,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000,44800.000000
mean,16.500000,247.436205,246.835558,246.329688,245.701674,245.073549,244.533929,244.036004,243.412344,242.875960,...,246.429643,246.892031,247.405937,247.706786,248.089397,248.448415,248.538929,248.674241,248.661674,248.791295
std,9.233196,14.309441,16.760471,18.577819,20.740866,22.711704,24.082806,24.765604,26.417180,27.622739,...,22.282476,20.319106,17.949820,16.793075,14.710475,12.063996,11.418188,10.434390,9.789630,8.774145
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,3.000000,3.000000,2.000000,3.000000,1.000000,3.000000,2.000000
25%,8.750000,244.000000,244.000000,244.000000,243.000000,243.000000,243.000000,243.000000,242.000000,242.000000,...,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000
50%,16.500000,249.000000,249.000000,249.000000,249.000000,248.000000,248.000000,248.000000,248.000000,247.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
75%,24.250000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,252.000000,252.000000,252.000000,...,254.000000,253.000000,253.000000,253.000000,254.000000,254.000000,254.000000,254.000000,254.000000,254.000000
max,32.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,...,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000


In [7]:
X = data.drop("Type",axis=1)
y = data["Type"]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [8]:
X_train = X_train/255.0
X_test = X_test/255.0

In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.decomposition import PCA

In [ ]:
svc_clf = SVC()
svc_clf.fit(X_train,y_train)
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train,y_train)
rnd_clf = RandomForestClassifier()
rnd_clf.fit(X_train,y_train)
voting_clf = VotingClassifier(estimators=[('m1',svc_clf),('m2',sgd_clf),('m3',rnd_clf)],voting='hard')
voting_clf.fit(X_train,y_train)
y_pred = voting_clf.predict(X_test)

In [ ]:
# Note: The following confusion matrix code is a remix of Scikit-Learn's 
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
# and Made with ML's introductory notebook - https://github.com/GokuMohandas/MadeWithML/blob/main/notebooks/08_Neural_Networks.ipynb
import itertools
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
             horizontalalignment="center",
             color="white" if cm[i, j] > threshold else "black",
             size=text_size)

In [ ]:
class_names = np.arange(1,33)
make_confusion_matrix(y_test,y_pred,classes=class_names)

In [ ]:
def Evaluate(X,y_true):
    y_pred_sgd = sgd_clf.predict(X)
    y_pred_rnd = rnd_clf.predict(X)
    y_pred_svc = svc_clf.predict(X)
    from sklearn.metrics import accuracy_score
    vot_clf_acc = accuracy_score(y_true,y_pred)
    sgd_clf_acc = accuracy_score(y_true,y_pred_sgd)
    rnd_clf_acc = accuracy_score(y_true,y_pred_rnd)
    svc_clf_acc = accuracy_score(y_true,y_pred_svc)
    precision = precision_score(y_true,y_pred)
    recall = recall_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    print("Accuracy:-")
    print(" ->SGDC - ",sgd_clf_acc)
    print(" ->Random Forest - ",rnd_clf_acc)
    print(" ->SVC - ",svc_clf_acc)
    print("-------------------")
    print("Voting classifier - ",vot_clf_acc)
    print("Precision : ",precision)
    print("Recall : ",recall)
    print("F1_score : ",f1)

In [ ]:
Evaluate(X_test_reduced,y_test)

In [ ]:
accuracy_score(y_test,y_pred)